In [1]:
%matplotlib qt

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt     
import matplotlib
import tensorflow as tf
import seaborn as sns

from core.simulation.validation import Gamma
from core.simulation.game import Phi


matplotlib.rcParams['figure.figsize'] = [20, 10]
sns.set(style="white")


In [2]:
# Import Data Frame

# Specify path
LOG_NUM =  6
PATH_TO_DF = r"C:\Users\lukas\OneDrive\Universität\Mathematik\Bachelorarbeit\log_dir\log_{}\SIMULATION_SUMMARY\simulation_summary_log_{}.pkl".format(LOG_NUM, LOG_NUM)
PATH_TO_MODEL = r"C:\Users\lukas\OneDrive\Universität\Mathematik\Bachelorarbeit\log_dir\log_{}\WEIGHTS\1".format(
    LOG_NUM
)

# Load data, calculate accuracy, remove greedy choices
df = pd.read_pickle(PATH_TO_DF)
df["difference_between_optimal_agent"] =df["optimal_final_cash_balance"]-df["agent_final_cash_balance"]
df_wo_greedy = df[df["greedy"]==0]

In [3]:
# Inspect the simulation results
df.tail()

,agent_choice_of_x1,agent_final_cash_balance,cumulative_liquidity_costs,episode,greedy,initial_state,optimal_choice_of_x1,optimal_final_cash_balance,difference_between_optimal_agent
4995,0.410556,6.826461,-0.010654,4995,1,"[[1.0], [1.0], [1.0], [0.0]]",NaN,NaN,NaN
4996,0.441901,11.222016,-0.025039,4996,0,"[[1.0], [1.0], [1.0], [0.0]]",NaN,NaN,NaN
4997,0.431346,5.167336,-0.009078,4997,0,"[[1.0], [1.0], [1.0], [0.0]]",NaN,NaN,NaN
4998,0.243401,6.676178,-0.017626,4998,1,"[[1.0], [1.0], [1.0], [0.0]]",NaN,NaN,NaN
4999,0.292115,8.192588,-0.008177,4999,1,"[[1.0], [1.0], [1.0], [0.0]]",NaN,NaN,NaN


In [5]:
# Metadata: alpha

alpha = 0.1
lower_bound=0.8
upper_bound=1.8
def get_mu_vola(lower_bound, upper_bound):
    mu = (lower_bound + upper_bound)/2 
    vola = (1/12)*(upper_bound - lower_bound)**2 
    return mu, vola

mu, vola = get_mu_vola(lower_bound, upper_bound)

print(f"Expected value of distribution {mu}, Volatility of distribution {vola}")

Expected value of distribution 1.3, Volatility of distribution 0.08333333333333333


In [6]:
# Calculate optimal choices 

initial_state = df.loc[0,"initial_state"]

def get_gamma_val(initial_state, alpha):
    x, y, S, t = initial_state[:, 0]
    return np.log((y * alpha) / (S) + 1) * (1 / alpha) + x

def get_optimal_x1(initial_state, alpha, mu, gamma_val):
    x,y,S,t = initial_state[:,0]
    x_star = (np.log(mu) + x * alpha) / (2 * alpha)
    if x_star < 0:
        return 0
    if (0 <= x_star) & (x_star <= gamma_val):
        return x_star
    else:
        return gamma_val
    
def max_return_optimal(initial_state, optimal_x1, mu, alpha):
    x,y,S,t = initial_state[:,0]
    return y-Phi(optimal_x1-x,alpha)*S - Phi(-optimal_x1,alpha)*S*mu


def optimal_vola(initial_state, optimal_x1, vola,alpha):
    x,y,S,t = initial_state[:,0]
    return (Phi(-optimal_x1,alpha)**2)*(S**2)*vola

gamma_val = get_gamma_val(initial_state, alpha)
optimal_x1 = get_optimal_x1(initial_state, alpha, mu, gamma_val)
return_optimal = max_return_optimal(initial_state, optimal_x1, mu,alpha)
vola_optimal = optimal_vola(initial_state, optimal_x1, vola,alpha)

print(f"Initial state:\n {initial_state},\n The optimal choice of x1: {optimal_x1},\n The expected return of optimal strategy: {return_optimal},\n The volatitlity of optimal strategy: {vola_optimal}")

Initial state:
 [[ 5.]
 [10.]
 [ 1.]
 [ 0.]],
 The optimal choice of x1: 3.811821322337455,
 The expected return of optimal strategy: 15.24060972188039,
 The volatitlity of optimal strategy: 0.8371258455171295


In [4]:
# Plot the responses for different events and regions
sns.lineplot(x="episode", y="agent_choice_of_x1",hue="greedy",
             data=df)

In [7]:
# Mooving average plot 1

X_AXIS = "episode"
Y_AXIS = "agent_choice_of_x1"

x_axis_data= df_wo_greedy[X_AXIS]
y_axis_data = df_wo_greedy[Y_AXIS]

MA_1 = 250
MA_2 = 1500
MA_3 = 3000

MA_1_wo_greedy = y_axis_data.rolling(window=MA_1).mean()
MA_2_wo_greedy = y_axis_data.rolling(window=MA_2).mean()
MA_3_wo_greedy = y_axis_data.rolling(window=MA_3).mean()


df_MA_wo_greedy =pd.DataFrame({X_AXIS:x_axis_data, Y_AXIS:y_axis_data, "MA_1":MA_1_wo_greedy, "MA_2":MA_2_wo_greedy, "MA_3":MA_3_wo_greedy})
sns.lineplot(x="episode", y=Y_AXIS, data=df_MA_wo_greedy, label="original series", alpha=0.5)
sns.lineplot(x="episode", y="MA_1", data=df_MA_wo_greedy, label="MA Factor: {}".format(MA_1))
sns.lineplot(x="episode", y="MA_2", data=df_MA_wo_greedy,label="MA Factor: {}".format(MA_2), c='r')
# sns.lineplot(x="episode", y="MA_3", data=df_MA_wo_greedy,label="MA Factor: {}".format(MA_3))
plt.title(Y_AXIS + " progression during training")
plt.xlabel(X_AXIS)
plt.ylabel(Y_AXIS)

C:\Users\lukas\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1725: UserWarning: Saw kwargs ['c', 'color'] which are all aliases for 'color'.  Kept value from 'color'
  seen=seen, canon=canonical, used=seen[-1]))
C:\Users\lukas\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1725: UserWarning: Saw kwargs ['c', 'color'] which are all aliases for 'color'.  Kept value from 'color'
  seen=seen, canon=canonical, used=seen[-1]))


Text(0, 0.5, 'agent_choice_of_x1')

In [16]:
# Mooving average plot 1

X_AXIS = "episode"
Y_AXIS_1 = "cumulative_liquidity_costs"
Y_AXIS_2 = "agent_choice_of_x1"
Y_AXIS_3 = "agent_final_cash_balance"

x_axis_data= df_wo_greedy[X_AXIS]
y_1_axis_data = df_wo_greedy[Y_AXIS_1]
y_2_axis_data = df_wo_greedy[Y_AXIS_2]
y_3_axis_data = df_wo_greedy[Y_AXIS_3]


MA_1 = 250
MA_2 = 250


ma_y_1_wo_greedy = y_1_axis_data.rolling(window=MA_1).mean()
ma_y_2_wo_greedy = y_2_axis_data.rolling(window=MA_1).mean()
ma_y_3_wo_greedy = y_3_axis_data.rolling(window=MA_2).mean()



df_MA_wo_greedy =pd.DataFrame({X_AXIS:x_axis_data, Y_AXIS_1:ma_y_1_wo_greedy, Y_AXIS_2:ma_y_2_wo_greedy, Y_AXIS_3:ma_y_3_wo_greedy})


sns.lineplot(x="episode", y=Y_AXIS_1, data=df_MA_wo_greedy, label="MA Factor: {}".format(MA_1) + Y_AXIS_1)
sns.lineplot(x="episode", y=Y_AXIS_2, data=df_MA_wo_greedy, label="MA Factor: {}".format(MA_1) + Y_AXIS_2)
sns.lineplot(x="episode", y=Y_AXIS_3, data=df_MA_wo_greedy, label="MA Factor: {}".format(MA_2) + Y_AXIS_3)

plt.title(Y_AXIS_1 + " and "+ Y_AXIS_2 +" and "+  Y_AXIS_3 + " progression during training")
plt.xlabel(X_AXIS)

Text(0.5, 0, 'episode')

In [8]:
# Final cash balance rolling volatility
X_AXIS = "episode"
Y_AXIS_1 = "agent_final_cash_balance"

x_axis_data= df_wo_greedy[X_AXIS]
y_1_axis_data = df_wo_greedy[Y_AXIS_1]

MA_1 = 250

ma_y_1_wo_greedy = y_1_axis_data.rolling(window=MA_1).std()
df_MA_wo_greedy =pd.DataFrame({X_AXIS:x_axis_data, Y_AXIS_1:ma_y_1_wo_greedy})


sns.lineplot(x="episode", y=Y_AXIS_1, data=df_MA_wo_greedy, label="Window size: {}".format(MA_1) + " of " + Y_AXIS_1)

plt.title(Y_AXIS_1 + " rolling std {} during training".format(MA_1))
plt.xlabel(X_AXIS)

Text(0.5, 0, 'episode')

In [8]:
# Plot 1 & 2
fig, ax =plt.subplots(2,1)
X_AXIS = "episode"
Y_AXIS_1 = "agent_choice_of_x1"
Y_AXIS_2 = "agent_final_cash_balance"

x_axis_data= df_wo_greedy[X_AXIS]
y_1_axis_data = df_wo_greedy[Y_AXIS_1]
y_2_axis_data = df_wo_greedy[Y_AXIS_2]

MA_1 = 1

ma_y_1_wo_greedy = y_1_axis_data.rolling(window=MA_1).mean()
ma_y_2_wo_greedy = y_2_axis_data.rolling(window=MA_1).mean()



df_MA_wo_greedy =pd.DataFrame({X_AXIS:x_axis_data, Y_AXIS_1:ma_y_1_wo_greedy, Y_AXIS_2:ma_y_2_wo_greedy})

sns.lineplot(x="episode", y=Y_AXIS_1, data=df_MA_wo_greedy, label="alpha=0.01", ax=ax[0])
sns.lineplot(x="episode", y=Y_AXIS_2, data=df_MA_wo_greedy, label="alpha=0.01".format(MA_1) , ax=ax[1])

ax[0].set(title="Strategy choice progression", xlabel="", ylabel="x_1 choice")
ax[1].set(title="Final cash balance progression", xlabel="Episode", ylabel="y_2 final cash")

ax[0].axhline(2.5, ls='--', label="Analytical solution, alpha=0.01", c="r")
ax[1].axhline(14.97, ls='--', label="Analytical solution, alpha=0.01", c="r")

fig.subplots_adjust(right=0.82, bottom=0.05)

ax[0].legend_.remove()
ax[1].legend(plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.))



TypeError: zip argument #2 must support iteration

In [9]:
# Plot 3 & 4
fig, ax =plt.subplots(2,1)
X_AXIS = "episode"
Y_AXIS_1 = "cumulative_liquidity_costs"
Y_AXIS_2 = "agent_final_cash_balance"

x_axis_data= df_wo_greedy[X_AXIS]
y_1_axis_data = df_wo_greedy[Y_AXIS_1]
y_2_axis_data = df_wo_greedy[Y_AXIS_2]

MA_1 = 250

ma_y_1_wo_greedy = y_1_axis_data.rolling(window=MA_1).mean()
ma_y_2_wo_greedy = y_2_axis_data.rolling(window=MA_1).std()



df_MA_wo_greedy =pd.DataFrame({X_AXIS:x_axis_data, Y_AXIS_1:ma_y_1_wo_greedy, Y_AXIS_2:ma_y_2_wo_greedy})

sns.lineplot(x="episode", y=Y_AXIS_1, data=df_MA_wo_greedy, label="alpha=0.01", ax=ax[0])
sns.lineplot(x="episode", y=Y_AXIS_2, data=df_MA_wo_greedy, label="alpha=0.01".format(MA_1) , ax=ax[1])
ax[0].axhline(-0.06, ls='--', label="Analytical solution, alpha=0.01", c="r")
ax[1].axhline(0.5, ls='--', label="Analytical solution, alpha=0.01", c="r")

ax[0].set(title="Cumulative liquidity costs progression", xlabel="", ylabel="Liqudity costs per episode")
ax[1].set(title="Rolling standard deviation of final cash balance", xlabel="Episode", ylabel="y_2 rolling standard deviation")
fig.subplots_adjust(right=0.82, bottom=0.05)
ax[1].legend_.remove()
ax[0].legend(plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.))


TypeError: zip argument #2 must support iteration

In [10]:
mean = df_wo_greedy["agent_final_cash_balance"][-8000:].mean()
std = df_wo_greedy["agent_final_cash_balance"][-8000:].std()

In [11]:
print(mean, std)

11.89837769472599 0.4287829393059271


In [12]:
df_wo_greedy["agent_choice_of_x1"][-8000:].mean()

3.1004449928700923

In [13]:
df_wo_greedy["cumulative_liquidity_costs"][-8000:].mean()

-4.0267488766195125

In [2]:
LOG_NUM = 50000
PATH_TO_DF_PERIOD = r"C:\Users\lukas\OneDrive\Universität\Mathematik\Bachelorarbeit\log_dir\log_{}\SIMULATION_SUMMARY\simulation_summary_transition_log_{}.pkl".format(LOG_NUM, LOG_NUM)
df_period = pd.read_pickle(PATH_TO_DF_PERIOD)


In [5]:
df_period.tail(20)

,episode,current_state,next_state
249980,49996,"[[1.0], [1.0], [1.0], [0.0]]","[[0.987], [1.0129992], [1.1612208], [1.0]]"
249981,49996,"[[0.987], [1.0129992], [1.1612208], [1.0]]","[[0.91117984], [1.1010097], [1.377283], [2.0]]"
249982,49996,"[[0.91117984], [1.1010097], [1.377283], [2.0]]","[[0.83881444], [1.2006413], [2.2978818], [3.0]]"
249983,49996,"[[0.83881444], [1.2006413], [2.2978818], [3.0]]","[[0.66436076], [1.6011658], [3.7843156], [4.0]]"
249984,49996,"[[0.66436076], [1.6011658], [3.7843156], [4.0]]","[[0.0], [4.1069837], [4.0670824], [5.0]]"
249985,49997,"[[1.0], [1.0], [1.0], [0.0]]","[[0.987], [1.0129992], [1.373555], [1.0]]"
249986,49997,"[[0.987], [1.0129992], [1.373555], [1.0]]","[[0.84147984], [1.2127337], [2.1682608], [2.0]]"
249987,49997,"[[0.84147984], [1.2127337], [2.1682608], [2.0]]","[[0.6821252], [1.557981], [2.0511808], [3.0]]"
249988,49997,"[[0.6821252], [1.557981], [2.0511808], [3.0]]","[[0.72782916], [1.4642125], [3.6560361], [4.0]]"
249989,49997,"[[0.72782916], [1.4642125], [3.6560361], [4.0]]","[[0.0], [4.115522], [6.348784], [5.0]]"


In [14]:
df_period.next_state[0]

array([[1.434     ],
       [0.56505686],
       [1.1274428 ],
       [1.        ]], dtype=float32)

In [13]:
df_period.next

[]

In [35]:
df_final = pd.DataFrame({"final_cash_balance": final_cash_balance})

[7.203621,
 3.6996355,
 8.384799,
 4.6671214,
 7.630034,
 8.026458,
 7.2547135,
 3.7093246,
 5.236111,
 2.691493]

In [117]:
final_cash_balance = [balance[1,0] for balance in df_period.next_state if balance[3,0] == 5]
final_cash_balance = final_cash_balance[0:30000]
print(len(final_cash_balance))
df_final = pd.DataFrame({"final_cash_balance": final_cash_balance})
df_final_rolling = df_final.rolling(window=100).mean()
df_final_rolling.plot(y="final_cash_balance",ylim=(0,4))

30000


In [80]:
price = [balance[2,0] for balance in df_period[3750:].next_state if balance[3,0] == 4]
df_price = pd.DataFrame({"price": price})
df_price_rolling = df_price.rolling(window=2000).mean()
df_price_rolling.plot(y="price")

In [122]:
price = [balance[0,0] for balance in df_period[0:10000].next_state if balance[3,0] == 1]
df_price = pd.DataFrame({"price": price})
df_price_rolling = df_price.rolling(window=100).mean()
df_price_rolling.plot(y="price", ylim=(0,1.5))

In [146]:
for period in range(0,6):
    price = [balance[1,0] for balance in df_period[40000:].next_state if balance[3,0] == period]
    df_price = pd.DataFrame({"price": price})
    print(f"Mean in {period}: {df_price.mean()}")


Mean in 0: price   NaN
dtype: float64
Mean in 1: price    1.043011
dtype: float64
Mean in 2: price    1.180493
dtype: float64
Mean in 3: price    1.365535
dtype: float64
Mean in 4: price    1.522037
dtype: float64
Mean in 5: price    3.423629
dtype: float64


In [145]:
np.std([np.random.uniform(0.8,1.8)*np.random.uniform(0.8,1.8)*np.random.uniform(0.8,1.8)*np.random.uniform(0.8,1.8) for x in range(500000)])

1.3134565220657508